In [1]:
from data import *
from encoders import *
from config import *
from models import *
print("using device: ", device)

import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


using device:  cpu


/Users/joyqiu/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### load local data files

In [2]:
# ugly engineering here
df = pd.read_csv('../../data/HR_events.csv')
df_y = pd.read_excel('../../data/PAS Challenge Outcome Data.xlsx', engine="calamine")
df_y = df_y[['VitalID', 'Died']]
df = df.merge(df_y, on='VitalID', how='left')
df['text'] = df['Died'].apply(lambda x: 'This infant will die in 7 days. ' if x == 1 else 'This infant will survive. ')
# df['text'] = df['text'] +' '+ df['event_description'].astype(str)
df['label'] = df.index.to_series()
# df_train, df_test = train_test_split(df, test_size=0.2)    

df_train, df_test = train_test_split(df, test_size=0.2, stratify=df['text'])    
# df_train['text'] = df_train['text'] +' '+ df_train['label'].astype(str)
# df_test['text'] = df_test['text'] +' '+ df_test['label'].astype(str)


### prepare datasets and model training

In [3]:
# text_encoder_name = 'dmis-lab/biobert-base-cased-v1.2'
text_encoder_name = 'sentence-transformers/all-mpnet-base-v2'

ts_f_train, tx_f_train, labels_train = get_features(df_train, text_encoder_name=text_encoder_name)
train_dataloader = CLIPDataset(ts_f_train, tx_f_train, labels_train).dataloader(batch_size=128)

ts_f_test, tx_f_test, labels_test = get_features(df_test, text_encoder_name=text_encoder_name)
test_dataloader = CLIPDataset(ts_f_test, tx_f_test, labels_test).dataloader(batch_size=128)


/Users/joyqiu/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
overwrite = True
model_path = './results/clip_hr_death.pth' 
loss_path = './results/clip_hr_death_losses.pth'
# Initialize model
model = CLIPModel(
        ts_dim=ts_f_train.shape[1],    # 32
        text_dim=tx_f_train.shape[1],  # 768
        projection_dim=128
    )

Layer (type:depth-idx)                   Param #
CLIPModel                                1
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       8,448
│    └─LeakyReLU: 2-2                    --
│    └─Linear: 2-3                       65,792
│    └─LeakyReLU: 2-4                    --
│    └─Linear: 2-5                       65,792
│    └─LeakyReLU: 2-6                    --
│    └─Linear: 2-7                       32,896
├─Sequential: 1-2                        --
│    └─Linear: 2-8                       196,864
│    └─LeakyReLU: 2-9                    --
│    └─Linear: 2-10                      131,584
│    └─LeakyReLU: 2-11                   --
│    └─Linear: 2-12                      131,328
│    └─LeakyReLU: 2-13                   --
│    └─Linear: 2-14                      65,792
│    └─LeakyReLU: 2-15                   --
│    └─Linear: 2-16                      32,896
Total params: 731,393
Trainable params: 731,393
Non-trainable params: 0


In [6]:

if overwrite or not os.path.exists(model_path):
    num_epochs = 10000
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.005)
    train_losses, test_losses = train(model, 
                                    train_dataloader,
                                    test_dataloader, 
                                    optimizer, 
                                    num_epochs, 
                                    device)
    # save model and losses
    torch.save(model.state_dict(), model_path)
    torch.save({
        'train_losses': train_losses,
        'test_losses': test_losses }, loss_path)
else:
    model.load_state_dict(torch.load(model_path))
    train_losses = torch.load(loss_path)['train_losses']
    test_losses = torch.load(loss_path)['test_losses']

    # plot losses
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(test_losses, label='Test Loss')
    plt.legend()
    plt.show()


  0%|          | 1/10000 [00:00<1:53:32,  1.47it/s]

Epoch [1/10000]
	Training Loss: 4.839318
	Testing Loss: 4.813427
	Learning Rate: 0.005000000


  0%|          | 2/10000 [00:01<1:57:13,  1.42it/s]

Epoch [2/10000]
	Training Loss: 4.802921
	Testing Loss: 4.813421
	Learning Rate: 0.005000000


  0%|          | 3/10000 [00:02<1:58:51,  1.40it/s]

...

Epoch [67/10000]
	Training Loss: 4.802918
	Testing Loss: 4.813420
	Learning Rate: 0.005000000


  1%|          | 68/10000 [00:46<1:52:05,  1.48it/s]

Epoch [68/10000]
	Training Loss: 4.802918
	Testing Loss: 4.813420
	Learning Rate: 0.005000000


  1%|          | 69/10000 [00:46<1:52:06,  1.48it/s]

: 

In [9]:

def get_prob(model, df_new, text_model_name):
    ts_f, tx_f, labels = get_features(df_new, text_encoder_name = text_model_name)
    dataloader = CLIPDataset(ts_f, tx_f, labels).dataloader(batch_size=1)
    model.eval()
    probs = []
    with torch.no_grad():
        for _, (ts_features, text_features, labels) in enumerate(dataloader):
            ts_features = ts_features.to(device)
            text_features = text_features.to(device)
            labels = labels.to(device)
            logits, _, _ = model(ts_features, text_features)
            prob = F.softmax(logits, dim=1)
            probs.append(prob)
    probs = torch.cat(probs, dim=0)
    return probs


In [12]:

df_new = df_test
df_new['text']= 'this infant will die in 7 days'

get_prob(model, df_new, text_model_name=text_encoder_name)



tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
      